In [1]:
import pandas as pd
from covalent_api import *
from config import API_KEY

In [2]:
pool_df = get_uniswapv3_pools(api_key=API_KEY, page_size=6000)
pool_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5696 entries, 0 to 5695
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dex_name                        5696 non-null   object 
 1   chain_name                      5696 non-null   object 
 2   chain_id                        5696 non-null   object 
 3   pool_contract_address           5696 non-null   object 
 4   pool_fee                        5696 non-null   object 
 5   pool_tick_spacing               5696 non-null   object 
 6   pool_deploy_time                5696 non-null   object 
 7   contract_address_token_0        5696 non-null   object 
 8   contract_name_token_0           5690 non-null   object 
 9   contract_ticker_symbol_token_0  5690 non-null   object 
 10  contract_decimals_token_0       5693 non-null   float64
 11  logo_url_token_0                5696 non-null   object 
 12  supports_erc_token_0            56

In [3]:
# don't want to make requests for pools without valid tokens
pool_df.dropna(subset=['contract_ticker_symbol_token_0', 'contract_ticker_symbol_token_1'],
               axis=0, inplace=True)

# coin/stable coin pairs with >50M 7D volume (as of 1/28)
top_pools = [
    'USDC-WETH-500',   # 9.36B 7D volume
    'USDC-WETH-3000',  # 1.42B 7D volume
    'WETH-USDT-500',   # 959M 7D volume
    'WETH-USDT-3000',  # 454M 7D volume
    'WBTC-USDC-3000',  # 297M 7D volume
    'DAI-WETH-500',    # 161M 7D volume
    'DAI-WETH-3000',   # 123M 7D volume
    'FRAX-WETH-3000',  # small pool for testing the our logic
]
pool_df['key'] = (
    pool_df['contract_ticker_symbol_token_0']
    + '-'
    + pool_df['contract_ticker_symbol_token_1']
    + '-'
    + pool_df['pool_fee']
)
pool_df = pool_df.loc[pool_df['key'].isin(top_pools), :].reset_index()
pool_df.drop('key', axis=1, inplace=True)

In [4]:
%%time
for i, row in pool_df.iterrows():
    pool_id = row['pool_contract_address']
    try:
        n = get_uniswapv3_counts(api_key=API_KEY, transaction='swaps', 
                                 pool_address=pool_id)         
    except (APIError, requests.ConnectionError) as e:
        print(f'Error querying swap counts for for {pool_id}.')
        print(f'{type(e).__name__}: {e}\n')
        n = np.nan
    pool_df.at[i, 'n_swaps'] = n

    try:
        n = get_uniswapv3_counts(api_key=API_KEY, transaction='liquidity', 
                                 pool_address=pool_id)  
    except (APIError, requests.ConnectionError) as e:
        print(f'Error querying liquidity transaction counts for for {pool_id}.')
        print(f'{type(e).__name__}: {e}\n')
        n = np.nan
    pool_df.at[i, 'n_liquidity'] = n

CPU times: user 301 ms, sys: 37.3 ms, total: 338 ms
Wall time: 3min 31s


In [5]:
# view the main results
cols = [
    'pool_contract_address',
    'contract_ticker_symbol_token_0',
    'contract_ticker_symbol_token_1',
    'n_swaps',
    'n_liquidity',
    'pool_deploy_time',
    'pool_fee'
]
pool_df.loc[:, cols].sort_values('n_swaps', ascending=False)

,pool_contract_address,contract_ticker_symbol_token_0,contract_ticker_symbol_token_1,n_swaps,n_liquidity,pool_deploy_time,pool_fee
2,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH,1246381.0,31421.0,2021-05-05T21:42:11Z,500
1,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT,735785.0,6421.0,2021-05-05T21:46:12Z,500
4,0x60594a405d53811d3bc4766596efd80fd545a270,DAI,WETH,295932.0,3515.0,2021-05-05T18:03:43Z,500
6,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH,194629.0,79804.0,2021-05-04T23:10:00Z,3000
5,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH,USDT,159328.0,29299.0,2021-05-05T16:37:08Z,3000
7,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,DAI,WETH,85097.0,11042.0,2021-05-04T20:16:08Z,3000
3,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,WBTC,USDC,59889.0,10913.0,2021-05-05T19:13:15Z,3000
0,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,FRAX,WETH,22.0,10.0,2021-05-06T03:25:59Z,3000


The USDC/ETH pool (0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640) is actually the largest pool, so it is a little concerning that we're encountering errors when trying to query the data. This was working a couple of days ago (1/25), so hopefully this is transitory and we are able to pull the data eventually.

In [6]:
# update the data types before outputting
dtypes = {
    'pool_fee': int,
    'pool_tick_spacing': int,
    'contract_decimals_token_0': int,
    'contract_decimals_token_1': int
}
pool_df = pool_df.astype(dtypes)
pool_df['pool_deploy_time'] = pd.to_datetime(pool_df['pool_deploy_time'])
pool_df.reset_index(drop=True, inplace=True)

pool_df.to_csv('./data/pool_info.csv', index=False)

In [ ]:
API_KEY = 'ckey_35d5e070f70b475babff7c4426d'
get_uniswapv3_counts(api_key=API_KEY, transaction='swaps', 
                     pool_address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')